<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 10.9MB/s 
     |████████████████████████████████| 890kB 19.4MB/s 
     |████████████████████████████████| 3.0MB 26.5MB/s 
     |████████████████████████████████| 36.2MB 82kB/s 
     |████████████████████████████████| 1.5MB 53.4MB/s 


In [11]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
import re
parsivar_tokenizer = Tokenizer()
parsivar_normalizer = Normalizer()

In [14]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()
print()

# **Tokenize**

### Example

In [22]:
tweet_example = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

tweet_example2 = """
 محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست

"""

tweet_example3 = """
سلام
خواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. 
چند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.

"""

In [23]:
input_sentence = tokenizer.tokenize(tweet_example3)
for token in input_sentence:
  print(token)

سلام
خواهر
من
فکر
کنم
دو
سالی
سرفه
امانش
را
بریده
بود
.
خیلی
کارها
کرد
برای
درمانش
اما
درست
نشد
که
نشد
.
کرونا
هم
که
شروع
شد
،
خیلی
نگرانش
بودیم
.
چند
روز
پیش
با
چند
تا
نیش
زنبور
عسل
به
اصطلاح
زنبور
درمانی
کرد
و
با
کمال
تعجب
دیگه
سرفه
نمی
کنه
.


In [24]:
for token in parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_example3)):
  print(token)

سلام
خواهر
من
فکر
کنم
دو
سالی
سرفه
امانش
را
بریده‌بود
.
خیلی
کارها
کرد
برای
درمانش
اما
درست
نشد
که
نشد
.
کرونا
هم
که
شروع
شد
،
خیلی
نگرانش‌بودیم
.
چند
روز
پیش
با
چند
تا
نیش
زنبور
عسل
به
اصطلاح
زنبور
درمانی
کرد
و
با
کمال
تعجب
دیگه
سرفه
نمی‌کنه
.


### Tokenize All tweets

In [17]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

--2020-10-05 05:33:53--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2051971 (2.0M) [text/plain]
Saving to: ‘extracted_data.txt’

extracted_data.txt  100%[===================>]   1.96M  9.44MB/s    in 0.2s    

2020-10-05 05:33:54 (9.44 MB/s) - ‘extracted_data.txt’ saved [2051971/2051971]



In [18]:
! head -n 40 extracted_data.txt

  سلام
خواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. 
چند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.

**************

ولادتِ حضرت امام موسی کاظمؑ 

**************

iran iranelection  - و صدای شیون فرزندانت که وحشت زده گریه می‌کردند؛ نوشته‌ی مهرزاد محب‌پور همسر مژگان عبادی بهای... 

**************

 Iran IranElection صبا محب‌پور، جوان محروم از تحصیل بهایی: «قانون، تنها عنوانی حک شده در کنار نام سرزمین من ایران است»

**************

  محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست

**************

 حسین باستانی یه مستند ساخته به اسم کودتای خزنده که بر اساس نوار درز کرده یه جلسه یه سری فرمانده‌ها در وسط جنگه با محسن رضایی (فرمانده کل سپاه تو اون زمان). تو اون جلسه فرمانده‌ها کلی شاکی‌ن از بی‌کفایتی یه سری از رده‌بالایی‌ها و باق

In [25]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'  سلام\nخواهر من فکر '

In [26]:
tweets = tweets_str.split("\n\n**************\n\n")
tweets[0], len(tweets)

('  سلام\nخواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. \nچند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.',
 6978)

In [27]:
len(tweets)

6978

In [28]:
len(set(tweets))

6543

In [ ]:
# tweets_tokens = []
# for tweet in tweets:
#   tweet_tokens = tokenizer.tokenize(tweet)
#   tweets_tokens.append(tweet_tokens)

In [29]:
tweets_tokens = []
for tweet in set(tweets):
  tweet_tokens = re.sub(' +', ' ',
                        tweet.replace('\n', ' ').
                        replace('.', ' ').
                        replace('،', ' ').
                        replace('!', ' ').
                        replace('?', ' ').
                        replace(':', ' ').
                        replace('؟', ' ').
                        replace('_', ' ').
                        replace('|', ' ').
                        replace('/', ' ').
                        replace('+', ' ').
                        replace('[', ' ').
                        replace(']', ' '))
  tweet_tokens = parsivar_tokenizer.tokenize_words(parsivar_normalizer.normalize(tweet_tokens))
  tweets_tokens.append(tweet_tokens)

In [30]:
tweets_tokens[1]

['تو',
 'فرشته',
 'الهیه',
 'تو',
 'صف',
 'بکنی',
 'چیز',
 'بهت',
 'نیمگن',
 'بابا',
 '))',
 'یهو',
 'از',
 '100',
 'می\u200cتونی',
 'بیای',
 '1',
 'کدوم',
 'سمتین',
 'مگه',
 '))))']

In [32]:
counter = 0
with open('persian-ner-twitter-data.txt', 'w') as file:
  for tweet_tokens in tweets_tokens:
    for token in tweet_tokens:
      if token != '':
        counter += 1
        file.write(token + '\tO\n')
    file.write('\n')
print(counter)

187405


In [33]:
! head -n 200 persian-ner-twitter-data.txt


تو	O
فرشته	O
الهیه	O
تو	O
صف	O
بکنی	O
چیز	O
بهت	O
نیمگن	O
بابا	O
))	O
یهو	O
از	O
100	O
می‌تونی	O
بیای	O
1	O
کدوم	O
سمتین	O
مگه	O
))))	O

چقدر	O
خوبست	O
که	O
حرف	O
حق	O
بزنیم	O
جالبست	O
ما	O
از	O
دروغ‌های	O
ج	O
ا	O
انتقاد	O
و	O
از	O
آن‌ها	O
ابراز	O
انزجار	O
می‌کنیم	O
اما	O
خودمان	O
مثل	O
آب	O
خوردن	O
دروغ	O
میگیم	O
و	O
به	O
دیگران	O
انگ	O
می‌زنیم	O
برای	O
مثال	O
نوری‌زاد	O
در	O
زندان	O
در	O
شرف	O
موت	O
است	O
اما	O
ما	O
اورا	O
مهره	O
ج	O
ا	O
مینامیم	O
آخر	O
وقتی	O
مرد	O
چه	O
پاداشی	O
را	O
می‌تواند	O
دریافت	O
کند	O
کمی	O
انصاف	O

فقط	O
موندم	O
چرا	O
جوادی	O
نشسته	O
داره	O
گوش	O
می‌ده	O

ایران	O
اینترنشنال	O
همین	O
چند	O
روز	O
پیش	O
یه	O
مستند	O
درباره	O
انتخابات	O
آمریکا	O
پخش	O
کرد	O
که	O
سری	O
هم	O
به	O
لاس‌وگاس	O
زد	O
و	O
به	O
یه	O
آموزشگاه	O
دیلری	O
رفت	O
مربی	O
و	O
صاحب	O
آموزشگاه	O
از	O
تاثیر	O
کرونا	O
رو	O
کسب	O
و	O
کارها	O
گفت	O
و	O
این	O
اعداد	O
رو	O
درباره	O
درآمد	O
و	O
دستمزد	O
دیلر‌ها	O
اون	O
ارائه	O
داد	O
گزارش	O
برای	O
یکهفته	O
پیشه	O

شهدای	O
مدافع	O
حرم	O
شهید	O
اینانلو	O

# **Model**

In [ ]:
from transformers import pipeline

In [ ]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet_example)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-organization',
  'index': 8,
  'score': 0.9995940327644348,
  'word': 'اتلتیکو'},
 {'entity': 'B-organization',
  'index': 13,
  'score': 0.9972712397575378,
  'word': 'بایرن'},
 {'entity': 'B-person',
  'index': 31,
  'score': 0.9915841817855835,
  'word': 'هاینکس'}]